In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [3]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

c:\Users\Ali\anaconda3\envs\ml\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ali\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassi

In [42]:
tokens = tokenizer.encode("I dont hate you", return_tensors="pt")

In [43]:
model_result = model(tokens)

In [44]:
model_result.logits

tensor([[ 0.3206,  0.8174, -1.2597]], grad_fn=<AddmmBackward0>)

In [45]:
int(torch.argmax(model_result.logits))+1

2

In [77]:
r = requests.get("https://www.yelp.com/biz/cucina-venti-restaurant-mountain-view-2")
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [79]:
soup

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#FF1A1A" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
      

In [80]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [81]:
def review_score(review):
      tokens = tokenizer.encode(review, return_tensors="pt")
      model_result = model(tokens)
      score = int(torch.argmax(model_result.logits))+1  
      if score == 1:
            return "Negative"
      elif score == 2:
            return "Neutral"
      else:
            return "Positive"
      

In [82]:
review_score(df['review'][0])

'Positive'

In [83]:
df['sentiment'] = df['review'].apply(lambda x: review_score(x[:512]))

In [66]:
df

,review,sentiment
0,Food and ambiance were awesome I ordered the b...,Positive
1,It was a delightful business lunch with my cus...,Positive
2,Exceptional food and service! I've brought fam...,Positive
3,"The best restaurant in Mountain. A hop, skip, ...",Positive
4,The food and the service was great. Very quiet...,Positive
5,"Horrendously overpriced. The food is good, the...",Negative
6,Simply delicious! All the plates I tried today...,Positive
7,Thank you Marge for dining at our location and...,Positive
8,I came in for a birthday dinner last week and ...,Positive
9,Thank you so much Humera for choosing to hold ...,Positive
